In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from collections import Counter
from ibm_watson import ToneAnalyzerV3
import ibm_watson
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from scikitplot.metrics import plot_roc_curve
from sklearn.metrics import roc_curve, roc_auc_score

In [14]:
##### Reading Datasets
all_reviews= pd.read_csv('./Data/Google Reviews.csv')  ## Google review dataset
#all_reviews = pd.read_csv('./Data/Play Store App reviews.csv')  ## App reviews from Google Play Store

In [15]:
final_df = all_reviews[['Final Coding','Word_count with stopwords', 'Review_length','NN', 'NNP', 'JJ', 'VB', 'RB', 'negative', 'neutral', 'positive','Tone_code']] #'Tone_code
final_df=final_df.replace(np.NaN,0)

In [16]:
X = final_df.drop('Final Coding',axis=1)
Y = final_df['Final Coding']

### Perform Over Sampling

In [17]:
from imblearn.over_sampling import RandomOverSampler

In [18]:
### Over Sampling, Used this technique
rus = RandomOverSampler(random_state=42)
x_rus, y_rus = rus.fit_resample(X, Y)

### 10 fold validation

In [19]:
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Naive Bayes', GaussianNB()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier(criterion="entropy",random_state=42)))
models.append(('Random Forest', RandomForestClassifier(n_estimators=70,criterion="entropy",random_state=42)))
models.append(('SVM', SVC()))
models.append(('Artificial Neural Network',MLPClassifier(hidden_layer_sizes=(70,70,70),max_iter=1000)))
models.append(('XGBoost',XGBClassifier()))
models.append(('AdaBoost',AdaBoostClassifier()))
SCV = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)

In [23]:
for name, model in models:
    
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    mcc_scores = []
    
    for train_index, test_index in SCV.split(x_rus,y_rus): 
        X_train, X_test, Y_train, Y_test = x_rus.iloc[train_index], x_rus.iloc[test_index], y_rus.iloc[train_index], y_rus.iloc[test_index]
        model.fit(X_train,Y_train)
        y_pred = model.predict(X_test)
        accuracy_scores.append(model.score(X_test,Y_test))
        precision_scores.append(precision_score(Y_test,y_pred))
        recall_scores.append(recall_score(Y_test,y_pred))
        f1_scores.append(f1_score(Y_test,y_pred))
        mcc_scores.append(matthews_corrcoef(Y_test,y_pred))
        
    print("Model is ",name)
    print("Average accuracy is: ", sum(accuracy_scores)/len(accuracy_scores))
    print("Average precision for decision classifier: ", sum(precision_scores)/len(precision_scores))
    print("Average recall for decision classifier: ", sum(recall_scores)/len(recall_scores))
    print("Average f1-score for decision classifier: ", sum(f1_scores)/len(f1_scores))
    print("Average MCC for decision classifier: ", sum(mcc_scores)/len(mcc_scores),"\n")
    
        
        
    
    

/Users/yasirmuhammad/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/yasirmuhammad/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/module

Model is  Logistic Regression
Average accuracy is:  0.8744588193685086
Average precision for decision classifier:  0.8896887801492541
Average recall for decision classifier:  0.8549442982640626
Average f1-score for decision classifier:  0.8719475621421549
Average MCC for decision classifier:  0.7495240959000202 

Model is  Naive Bayes
Average accuracy is:  0.8394812243057066
Average precision for decision classifier:  0.9091456825850359
Average recall for decision classifier:  0.7543682488940402
Average f1-score for decision classifier:  0.8245286743191249
Average MCC for decision classifier:  0.6890461393952803 

Model is  KNN
Average accuracy is:  0.9852754882524148
Average precision for decision classifier:  1.0
Average recall for decision classifier:  0.9705513259313612
Average f1-score for decision classifier:  0.9850481757129199
Average MCC for decision classifier:  0.9709850732796891 

Model is  Decision Tree
Average accuracy is:  0.9937331589825789
Average precision for decisio

KeyboardInterrupt: 